In [ ]:
import requests
import pandas as pd
from newspaper import Article
from tqdm import tqdm  # progress bar for nicer output

Status Code: 200
Content-Type: text/html; charset=utf-8
First 500 characters of content:
 Your query was too short or too long.



In [ ]:
# GDELT Doc API endpoint
url = "https://api.gdeltproject.org/api/v2/doc/doc"
params = {
    "query": "united states",
    "mode": "artlist",
    "maxrecords": 250,  # slightly higher in case non-US articles are included
    "format": "json",
    "sort": "datedesc",
    "sourcecountry": "US"
}

response = requests.get(url, params=params)

if response.status_code != 200:
    print("Error:", response.status_code)
    print(response.text)
else:
    data = response.json()
    articles = data.get("articles", [])
    
    # Convert to DataFrame
    df = pd.DataFrame(articles)

    # Filter only rows where sourcecountry is "United States"
    df_us = df[df["sourcecountry"] == "United States"].head(100)

    # Show selected columns
    print(df_us[["title", "url", "seendate", "sourcecountry"]])

    # Save to CSV
    df_us.to_csv("gdelt_us_articles.csv", index=False)
    print("Saved to gdelt_us_articles.csv ✅")


                                                 title  \
1    From Cell to System , Explore New Insights in ...   
2    Expo Contratista Names QXO as Exhibitor at 202...   
4    Natrona County Arrest Log ( 07 / 25 / 25 – 07 ...   
6    Vinitaly . USA , the Largest Italian Wine Even...   
7    Werba Rubin Papier Wealth Management Has $9 . ...   
..                                                 ...   
236   Flexible Work Benefits All Women , Not Just Moms   
238  Salmonella Outbreak Leads To Deepbrand Bean Pr...   
240      Lazy River in Iowa Ranked Top 3 in the Nation   
243  NAMI Rome to present special program Thursday ...   
248          Streaming Stocks To Watch Now – July 26th   

                                                   url          seendate  \
1    https://www.newswise.com/articles/from-cell-to...  20250728T190000Z   
2    http://www.hawaiitelegraph.com/news/278473065/...  20250728T190000Z   
4    https://mycountry955.com/ixp/101/p/natrona-cou...  20250728T190000Z   

In [ ]:
url = "https://www.sbsun.com/2025/07/28/trump-gaza-food/"
article = Article(url)
article.download()
article.parse()

print(article.title)
print(article.text)

Trump says he wants Netanyahu to ‘make sure they get the food’ in Gaza amid humanitarian crisis
By MICHELLE L. PRICE

WASHINGTON (AP) — President Donald Trump on Monday expressed concern over the worsening humanitarian situation in Gaza and urged Israel to get people food, seemingly recalibrating his stance on Gaza as images of emaciated children have sparked renewed worries about hunger in the war-torn territory.

Trump, speaking in Scotland on Monday, said the U.S. and other nations are giving money and food to Gaza but that Israeli Prime Minister Benjamin Netanyahu has “got to sort of like run it.”

“I want him to make sure they get the food,” Trump said. “I want to make sure they get the food.”

Trump’s comments seemed to result from the images in recent days of the worsening hunger crisis in Gaza and were more urgent than the resigned message he had about the 21-month Israel-Hamas war last week, when ceasefire talks derailed. His remarks Monday also marked a new divergence from Ne

In [ ]:
# Step 1: Query GDELT Doc API for recent US articles
url = "https://api.gdeltproject.org/api/v2/doc/doc"
params = {
    "query": "united states",
    "mode": "artlist",
    "maxrecords": 250,
    "format": "json",
    "sort": "datedesc",
    "sourcecountry": "US"
}

response = requests.get(url, params=params)
if response.status_code != 200:
    print("Error:", response.status_code)
    print(response.text)
    exit()

data = response.json()
articles = data.get("articles", [])

# Step 2: Convert to DataFrame
df = pd.DataFrame(articles)

# Filter only "United States" sourcecountry and limit to 100 articles
df_us = df[df["sourcecountry"] == "United States"].head(100)

# Step 3: Extract article content using newspaper3k
def extract_content(article_url):
    try:
        article = Article(article_url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        return None  # or you can return str(e) to debug failures

print("Extracting article content for each URL... This may take some time.")
df_us["content"] = [extract_content(url) for url in tqdm(df_us["url"])]

# Optional: Drop articles with no content
df_us = df_us.dropna(subset=["content"])

# Step 4: Save to CSV
df_us.to_csv("gdelt_us_articles_with_content.csv", index=False)
print("Saved articles with content to gdelt_us_articles_with_content.csv")


Extracting article content for each URL... This may take some time.


100%|██████████| 92/92 [00:48<00:00,  1.89it/s]

Saved articles with content to gdelt_us_articles_with_content.csv
